# Learning Bayesian Networks from Data


Previous notebooks showed how Bayesian networks economically encode a probability distribution over a set of variables, and how they can be used e.g. _to predict variable states. This section will be about obtaining a Bayesian network, given a set of sample data_. Learning a Bayesian network can be split into two problems:

 **Parameter learning:** Given a set of data samples and a DAG that captures the dependencies between the variables, estimate the (conditional) probability distributions of the individual variables.

 **Structure learning:** Given a set of data samples, estimate a DAG that captures the dependencies between the variables.

This notebook aims to illustrate how parameter learning can be done with `pgmpy`.
Currently, the library supports:
 - Parameter learning for *discrete* nodes:
   - Maximum Likelihood Estimation
   - Bayesian Estimation with Smoothing
   - Expectation Maximization (EM)


## 1. Maximum Likelihood Estimation

Suppose we have the following data:

In [15]:
import pandas as pd
data = pd.DataFrame(data={'fruit': ["banana", "apple", "banana", "apple", "banana","apple", "banana",
                                    "apple", "apple", "apple", "banana", "banana", "apple", "banana",],
                          'tasty': ["yes", "no", "yes", "yes", "yes", "yes", "yes",
                                    "yes", "yes", "yes", "yes", "no", "no", "no"],
                          'size': ["large", "large", "large", "small", "large", "large", "large",
                                    "small", "large", "large", "large", "large", "small", "small"]})
print(data)

     fruit tasty   size
0   banana   yes  large
1    apple    no  large
2   banana   yes  large
3    apple   yes  small
4   banana   yes  large
5    apple   yes  large
6   banana   yes  large
7    apple   yes  small
8    apple   yes  large
9    apple   yes  large
10  banana   yes  large
11  banana    no  large
12   apple    no  small
13  banana    no  small


We know that the variables relate as follows:

In [16]:
!pip install pgmpy
from pgmpy.models import DiscreteBayesianNetwork

model = DiscreteBayesianNetwork([('fruit', 'tasty'), ('size', 'tasty')])  # fruit -> tasty <- size

Parameter learning is the task to estimate the values of the conditional probability distributions (CPDs), for the variables `fruit`, `size`, and `tasty`.

#### State counts
To make sense of the given data, we can start by counting how often each state of the variable occurs. If the variable is dependent on parents, the counts are done conditionally on the parents states, i.e. separately for each parent configuration:

In [17]:
from pgmpy.estimators import ParameterEstimator
pe = ParameterEstimator(model, data)
print("\n", pe.state_counts('fruit'))  # unconditional
print("\n", pe.state_counts('tasty'))  # conditional on fruit and size


         count
fruit        
apple       7
banana      7

 fruit apple       banana      
size  large small  large small
tasty                         
no      1.0   1.0    1.0   1.0
yes     3.0   2.0    5.0   0.0


We can see, for example, that as many apples as bananas were observed and that `5` large bananas were tasty, while only `1` was not.

#### Maximum Likelihood Estimation

A natural estimate for the CPDs is to simply use the *relative frequencies*, with which the variable states have occured. We observed `7 apples` among a total of `14 fruits`, so we might guess that about `50%` of `fruits` are `apples`.

This approach is *Maximum Likelihood Estimation (MLE)*. According to MLE, we should fill the CPDs in such a way, that $P(\text{data}|\text{model})$ is maximal. This is achieved when using the *relative frequencies*. See [1], section 17.1 for an introduction to ML parameter estimation. `pgmpy` supports MLE as follows:

In [18]:
from pgmpy.estimators import MaximumLikelihoodEstimator
mle = MaximumLikelihoodEstimator(model, data)
print(mle.estimate_cpd('fruit'))  # unconditional
print(mle.estimate_cpd('tasty'))  # conditional

+---------------+-----+
| fruit(apple)  | 0.5 |
+---------------+-----+
| fruit(banana) | 0.5 |
+---------------+-----+
+------------+--------------+-----+---------------+
| fruit      | fruit(apple) | ... | fruit(banana) |
+------------+--------------+-----+---------------+
| size       | size(large)  | ... | size(small)   |
+------------+--------------+-----+---------------+
| tasty(no)  | 0.25         | ... | 1.0           |
+------------+--------------+-----+---------------+
| tasty(yes) | 0.75         | ... | 0.0           |
+------------+--------------+-----+---------------+


- `mle.estimate_cpd(variable)` computes the state counts and divides each cell by the (conditional) sample size.
- The `mle.get_parameters()`-method returns a list of CPDs for all variables of the model.

- The built-in `fit()`-method of `BayesianNetwork` provides more convenient access to parameter estimators:


In [19]:
# Calibrate all CPDs of `model` using MLE:
model.fit(data, estimator=MaximumLikelihoodEstimator)


While very straightforward, the ML estimator has the problem of *overfitting* to the data. In above CPD, the probability of a large banana being tasty is estimated at `0.833`, because `5` out of `6` observed large bananas were tasty. Fine. But note that the probability of a small banana being tasty is estimated at `0.0`, because we  observed only one small banana and it happened to be not tasty. But that should hardly make us certain that small bananas aren't tasty!
We simply do not have enough observations to rely on the observed frequencies. If the observed data is not representative for the underlying distribution, ML estimations will be extremly far off.

When estimating parameters for Bayesian networks, lack of data is a frequent problem. Even if the total sample size is very large, the fact that state counts are done conditionally for each parents configuration causes immense fragmentation. If a variable has 3 parents that can each take 10 states, then state counts will be done separately for `10^3 = 1000` parents configurations. This makes MLE very fragile and unstable for learning Bayesian Network parameters. A way to mitigate MLE's overfitting is *Bayesian Parameter Estimation*.

## 2. Bayesian Parameter Estimation with Smoothing

The Bayesian Parameter Estimator starts with already existing prior CPDs, that express our beliefs about the variables *before* the data was observed. Those "priors" are then updated, using the state counts from the observed data. See [1], Section 17.3 for a general introduction to Bayesian estimators.

One can think of the priors as consisting in *pseudo state counts*, that are added to the actual counts before normalization.
Unless one wants to encode specific beliefs about the distributions of the variables, one commonly chooses uniform priors, i.e. ones that deem all states equiprobable.

A very simple prior is the so-called `K2` prior (_Laplace smoothing_), which simply adds `1` to the count of every single state.
A somewhat more sensible choice of prior is `BDeu` (Bayesian Dirichlet equivalent uniform prior). For `BDeu` we need to specify an _equivalent sample size_ `N` and then the pseudo-counts are the equivalent of having observed `N` uniform samples of each variable (and each parent configuration). In `pgmpy`:




In [20]:
from pgmpy.estimators import BayesianEstimator
est = BayesianEstimator(model, data)

print(est.estimate_cpd('tasty', prior_type='BDeu', equivalent_sample_size=10))

+------------+---------------------+-----+---------------------+
| fruit      | fruit(apple)        | ... | fruit(banana)       |
+------------+---------------------+-----+---------------------+
| size       | size(large)         | ... | size(small)         |
+------------+---------------------+-----+---------------------+
| tasty(no)  | 0.34615384615384615 | ... | 0.6428571428571429  |
+------------+---------------------+-----+---------------------+
| tasty(yes) | 0.6538461538461539  | ... | 0.35714285714285715 |
+------------+---------------------+-----+---------------------+


The estimated values in the CPDs are now more conservative. In particular, the estimate for a small banana being not tasty is now around `0.64` rather than `1.0`. Setting `equivalent_sample_size` to `10` means that for each parent configuration, we add the equivalent of 10 uniform samples (here: `+5` small bananas that are tasty and `+5` that aren't).

`BayesianEstimator`, too, can be used via the `fit()`-method. Full example:

In [21]:
import numpy as np
import pandas as pd
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.estimators import BayesianEstimator

# generate data
data = pd.DataFrame(np.random.randint(low=0, high=2, size=(5000, 4)), columns=['A', 'B', 'C', 'D'])
model = DiscreteBayesianNetwork([('A', 'B'), ('A', 'C'), ('D', 'C'), ('B', 'D')])

model.fit(data, estimator=BayesianEstimator, prior_type="BDeu") # default equivalent_sample_size=5
for cpd in model.get_cpds():
    print(cpd)


+------+--------+
| A(0) | 0.4996 |
+------+--------+
| A(1) | 0.5004 |
+------+--------+
+------+---------------------+---------------------+
| A    | A(0)                | A(1)                |
+------+---------------------+---------------------+
| B(0) | 0.49400119976004797 | 0.49720503094430024 |
+------+---------------------+---------------------+
| B(1) | 0.505998800239952   | 0.5027949690556998  |
+------+---------------------+---------------------+
+------+--------------------+-----+---------------------+
| A    | A(0)               | ... | A(1)                |
+------+--------------------+-----+---------------------+
| D    | D(0)               | ... | D(1)                |
+------+--------------------+-----+---------------------+
| C(0) | 0.4927841250751654 | ... | 0.47896473499690656 |
+------+--------------------+-----+---------------------+
| C(1) | 0.5072158749248347 | ... | 0.5210352650030934  |
+------+--------------------+-----+---------------------+
+------+---------

## 3. Expection Maximization (EM) Algorithm

For this example, we simulate some data from the [alarm model](https://www.bnlearn.com/bnrepository/discrete-medium.html#alarm) and use it to learn back the model parameters. In this example, we simply use the structure to the _alarm model_.

In [22]:
from pgmpy.utils import get_example_model
from pgmpy.models import DiscreteBayesianNetwork

# Load the alarm model and simulate data from it.
alarm_model = get_example_model(model="alarm")
samples = alarm_model.simulate(n_samples=int(1e3))

print(samples.head())

# Define a new model with the same structure as the alarm model.
new_model = DiscreteBayesianNetwork(ebunch=alarm_model.edges())

  0%|          | 0/37 [00:00<?, ?it/s]

  LVEDVOLUME   PVSAT VENTMACH    FIO2   PRESS    PCWP VENTLUNG CATECHOL  \
0     NORMAL     LOW   NORMAL  NORMAL    HIGH  NORMAL     ZERO     HIGH   
1     NORMAL  NORMAL   NORMAL     LOW  NORMAL  NORMAL      LOW   NORMAL   
2     NORMAL     LOW   NORMAL  NORMAL    HIGH  NORMAL   NORMAL   NORMAL   
3     NORMAL     LOW   NORMAL  NORMAL    HIGH  NORMAL     ZERO     HIGH   
4       HIGH  NORMAL   NORMAL  NORMAL    HIGH    HIGH     ZERO     HIGH   

      PAP STROKEVOLUME  ... VENTTUBE VENTALV INTUBATION ANAPHYLAXIS LVFAILURE  \
0  NORMAL       NORMAL  ...      LOW    ZERO     NORMAL       FALSE     FALSE   
1  NORMAL       NORMAL  ...     HIGH    HIGH     NORMAL       FALSE     FALSE   
2  NORMAL       NORMAL  ...     ZERO  NORMAL     NORMAL       FALSE     FALSE   
3  NORMAL       NORMAL  ...      LOW    ZERO     NORMAL       FALSE     FALSE   
4  NORMAL       NORMAL  ...      LOW    ZERO     NORMAL       FALSE     FALSE   

  HISTORY     TPR   SHUNT MINVOL      BP  
0   FALSE  NORMAL  

The _Expectation Maximization_ (EM) estimator can work in the case when latent variables are present in the model. To simulate this scenario, we will specify some of the variables in our `new_model` as latents and drop those variables from samples to simulate missing data.

In [23]:
model_latent = DiscreteBayesianNetwork(alarm_model.edges(), latents={'HISTORY', 'CVP'})
samples_latent = samples.drop(['HISTORY', 'CVP'], axis=1)

In [24]:
from pgmpy.estimators import ExpectationMaximization as EM
em_est = EM(model=model_latent, data=samples_latent)
# Generate HTML table for each CPD
#em_est.get_parameters()
# Initialize the EM estimator
em_est = EM(model=model_latent, data=samples_latent)

# Iterate through CPDs generated by the model
for cpd in em_est.get_parameters():
    print(f"Conditional Probability Distribution for {cpd.variable}:")
    print(cpd)

  0%|          | 0/100 [00:00<?, ?it/s]

Conditional Probability Distribution for LVEDVOLUME:
+--------------------+-----+-------------------+
| HYPOVOLEMIA        | ... | HYPOVOLEMIA(TRUE) |
+--------------------+-----+-------------------+
| LVFAILURE          | ... | LVFAILURE(TRUE)   |
+--------------------+-----+-------------------+
| LVEDVOLUME(HIGH)   | ... | 0.0               |
+--------------------+-----+-------------------+
| LVEDVOLUME(LOW)    | ... | 1.0               |
+--------------------+-----+-------------------+
| LVEDVOLUME(NORMAL) | ... | 0.0               |
+--------------------+-----+-------------------+
Conditional Probability Distribution for PVSAT:
+---------------+-----+----------------------+
| FIO2          | ... | FIO2(NORMAL)         |
+---------------+-----+----------------------+
| VENTALV       | ... | VENTALV(ZERO)        |
+---------------+-----+----------------------+
| PVSAT(HIGH)   | ... | 0.0                  |
+---------------+-----+----------------------+
| PVSAT(LOW)    | ... | 0.99541